# Shawn's Workspace

In [9]:
from modsim import *
from math import *
#takes velocity

In [82]:
m = UNITS.meter
km = UNITS.kilometer
s = UNITS.second
kg = UNITS.kilogram
N = kg * m /(s**2)
kN = kg * m /(s**2) * 1000
G = 6.67408 * (10**-11) * m**3 * kg**-1 * s**-2

mars = System(x= 0 * m,
              y= 0 * m,
              radius = 3396200 * m,
              mass = .64171 * (10**24) * kg
              orbital_speed = 24080 * m/s 
                )

rocket = System(x= -1 * mars.radius,
                y= 0 * m,
                dry_mass = 1433788 * kg,
                rho = 0.02 * kg/m**3, #needs function to update
                h = 0 * m, #height
                thrust = 22800000 * N,
                fuel_init = 1000000 * kg
               )


SyntaxError: invalid syntax (<ipython-input-82-f1f2c6ef6756>, line 13)

In [92]:
sun = System(x = 0 * m, #i'm using conditions instead of systems to reduce the am't of systems we have
             y = 0 * m, # I converted sun and mars to systems for now
                        # because something wasn't working with the make system function
             mass = 1.9891e30 * kg             
            )

mars = System(orbital_radius = 227.9e9 * m,
              radius = 3396200 * m,
              mass = .64171e24 * kg,
              orbital_speed = 24080 * m/s 
                )

saturn = Condition(orbital_radius = 1.429e12 * m,
                mass = 5.683e26 * kg,
                orbital_speed = 9680 *m/s,
                radius= 58.232e6 * m
                )

titan = Condition(radius = 2575e3 *m,
               mass = 1.3455e23 * kg,
               orbital_speed = 5570 *m/s,
               orbital_radius = 1221865e3 * m
              )

rocket = Condition(orbital_radius = 0 *m,
                dry_mass = 1433788 * kg,
                rho = 0.02 * kg/m**3, #needs function to update
                thrust = 22800000 * N,
                fuel_init = 1000000 * kg
               )


###
#sun.mass, saturn.radius, mars.radius
#vp = (2*G*sun.mass*(saturn.radius/(mars.radius*(mars.radius+saturn.radius))))**(1/2)

#dv1 = vp - mars.orbital_speed
#va = vp*(mars.radius/saturn.radius)
#dV2 = saturn.orbital_speed - va

#dv = g*I*ln(mass_initial/mass_final)

In [67]:
def make_system_planet(condition,theta): #in degrees
    
    unpack(condition)
    
    theta1 = np.deg2rad(theta*UNITS.degree)
    
    x,y = pol2cart(theta1,orbital_radius)
    
    vx,vx = pol2cart(theta1+.5*pi*UNITS.radian,orbital_speed)
    
    init = State(x=x,y=y,vx=vx,vy=vy)
    
    return System(init=init,mass=mass)
    
def make_system_titan(condition,system,theta):
    
    unpack(condition)
    
    theta1 = np.deg2rad(theta*UNITS.degree)
    
    a = Vector(system.init.x,system.init.y)
    
    x,y = pol2cart(theta1,orbital_radius)
    
    vx,vx = pol2cart(theta1+.5*pi*UNITS.radian,orbital_speed)
    
    init = State(x=x,y=y,vx=vx,vy=vy)
    
    return System(init=init,mass=mass)

    
def slope_func(state,t,system):
    x, y, vx, vy = state
    unpack(system)
    
    h = height_from_mars(x,y)
    a_grav = calc_grav(h)
    
    v = Vector(vx,vy)
    
    f_drag = force_of_drag(v.mag,h)
    a_drag = d_drag/mass
    
    a_thrust = accel_thrust(system,v,t)
    
    a = a_grav + a_drag + a_thrust
    
    return v.x, v.y, a.x, a.y



# This is the start of the things I've done today

In [62]:
thrust_magnitude = 934000 * N,
flow_rate = -273.6 * kg/s
mass_initial = 108000 * kg
g = 9.81 * m * s**-2
I = 307.4 * s #Merlin in vacuum from https://www.reddit.com/r/spacex/comments/3lsm0q/f9ft_vs_f9v11_fuel_mass_flow_rate_isp/

def calc_dv(burn_time):
    """ 
    Calculates a change in velocity, taking burn time of thrusters as input
    implies a constant thrust, defined above
    refers to STAGE TWO of Spacex Falcon 9, link to source above
    """
    new_mass = mass_initial + burn_time*s*flow_rate
    """
    Calculates change in mass of fuel over time 
    references flow rate from source
    """
    dv = g*I*log(mass_initial/new_mass)
    return dv
    
calc_dv(100)


<Quantity(880.9648437806447, 'meter / second')>

In [95]:
def calc_radii(dv):
    vp =  mars.orbital_speed + dv
    """
    Takes the change in velocity from the previous function
    Adds change in velocity to speed of mars, gives us speed of rocket as it orbits around the sun
    Puts this velocity into a magic sauce equation to calculate the distance 
    from the sun to the perigee and apogee, labelled r1 and r2 respectively
    This is the goal but the unit's aren't working, rip
    """
    f = G * sun.mass*mars.radius - (vp**2)*(mars.radius**2)/ ((vp**2)*(mars.radius)-2*G*sun.mass)
    print(f)
    r1 = mars.radius
    r2 = 2*f+r1
    return r1, r2

calc_radii(1000)
    




DimensionalityError: Cannot convert from 'meter / second' to 'dimensionless'